In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
ruta = '/content/drive/MyDrive/Tesis_MSDS/Horario_LEV.txt'

# Cargar el archivo
# horario = pd.read_csv(ruta+"/Horario_LEV.txt")

### 📥 1. Cargar cromosoma desde archivo `.txt`

En este bloque se carga el cromosoma almacenado en un archivo `.txt`, transformándolo en un DataFrame de Pandas con columnas organizadas por nivel, aula, día, hora, materia y docente. Se normalizan los tipos de datos y se eliminan espacios innecesarios.


In [ ]:
import pandas as pd

# --- 1. Cargar cromosoma ---
cromosoma = []
with open(ruta, "r", encoding="utf-8") as f:
    for linea in f:
        if linea.strip():
            cromosoma.append(eval(linea.strip()))

df = pd.DataFrame(cromosoma, columns=["Nivel", "Aula", "Día", "Hora", "Materia", "Docente"])
df["Hora"] = df["Hora"].astype(int)
for col in ["Nivel", "Aula", "Día", "Materia", "Docente"]:
    df[col] = df[col].astype(str).str.strip()

### 🧩 2. Definición de bloques especiales

Función `tipo_bloque_especial` que determina si un bloque de horario debe ser marcado como `RECESO`, `ALMUERZO`, `EVALEV` o `CLUBS`, en función del nivel, día y franja horaria. Esta lógica se utiliza para agregar automáticamente estas actividades en el horario generado.


In [ ]:
# --- 2. Bloques especiales ---
def tipo_bloque_especial(nivel, dia, hora):
    if nivel in ["1EGB", "2EGB", "3EGB"] and hora == 5:
        return "RECESO"
    if nivel not in ["1EGB", "2EGB", "3EGB"] and hora == 6:
        return "RECESO"
    if nivel in ["1EGB", "2EGB", "3EGB", "4EGB", "5EGB", "6EGB", "7EGB"] and hora in [11, 12]:
        return "ALMUERZO"
    if nivel in ["8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"] and hora in [13, 14]:
        return "ALMUERZO"
    if dia == "Miércoles" and hora in [1, 2]:
        return "EVALEV"
    if nivel == "1EGB" and ((dia == "Lunes" and hora in [15, 16]) or (dia == "Jueves" and hora in [13, 14])):
        return "CLUBS"
    if nivel in ["2EGB", "3EGB", "4EGB", "5EGB", "6EGB", "7EGB"] and dia in ["Martes", "Jueves"] and hora in [15, 16]:
        return "CLUBS"
    if nivel in ["8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"] and dia in ["Miércoles", "Viernes"] and hora in [15, 16]:
        return "CLUBS"
    return None

### 🔤 3. Función de escape para caracteres especiales de LaTeX

La función `latex_escape` reemplaza los caracteres conflictivos como `\`, `_`, `%`, etc., por versiones seguras compatibles con LaTeX. Esto asegura que los nombres de materias y docentes no causen errores de compilación.


In [ ]:
# --- 3. Escapar caracteres especiales LaTeX ---
def latex_escape(texto):
    return (
        texto.replace("\\", "\\textbackslash{}")
             .replace("&", "\\&")
             .replace("%", "\\%")
             .replace("$", "\\$")
             .replace("#", "\\#")
             .replace("_", "\\_")
             .replace("{", "\\{")
             .replace("}", "\\}")
             .replace("~", "\\textasciitilde{}")
             .replace("^", "\\^{}")
    )

### 🧾 4. Función `generar_tabla_latex`: construcción del horario en formato LaTeX

Esta función toma como entrada un nivel, un aula y el DataFrame con asignaciones, y genera automáticamente una tabla en código LaTeX:

- Define la estructura de días y horas.
- Rellena los bloques especiales y las horas de tutoría.
- Inserta las asignaciones de materias con nombre de docente.
- Aplica coloreado condicional con `\cellcolor`.
- Devuelve un bloque completo de tabla listo para insertar en documentos LaTeX usando `landscape`, `makecell` y `tabular`.

Incluye etiquetas (`\label`) y estilos gráficos como `arrayrulewidth`, `rowcolor` y `caption`.


In [ ]:
# --- 4. Función para generar tabla con formato final ---
def generar_tabla_latex(nivel, aula, df):
    dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]
    franjas = {
        "Act. Inicial": "7h30 a 7h45", 1: "7h45 a 8h15", 2: "8h15 a 8h45", 3: "8h45 a 9h15",
        4: "9h15 a 9h45", 5: "9h45 a 10h15", 6: "10h15 a 10h45", 7: "10h45 a 11h15",
        8: "11h15 a 11h45", 9: "11h45 a 12h15", 10: "12h15 a 12h45", 11: "12h45 a 13h15",
        12: "13h15 a 13h45", 13: "13h45 a 14h15", 14: "14h15 a 14h45", 15: "14h45 a 15h15",
        16: "15h15 a 15h45", "Act. Final": "15h45 a 16h00"
    }
    colores = {
        "TUTORIA": "purple!20",
        "RECESO": "cyan!20",
        "ALMUERZO": "teal!20",
        "CLUBS": "orange!20",
        "EVALEV": "yellow!30"
    }

    horas = ["Act. Inicial"] + list(range(1, 17)) + ["Act. Final"]
    tabla = pd.DataFrame(index=horas, columns=dias)

    # Rellenar tabla con bloques especiales y TUTORIA
    for h in horas:
        for d in dias:
            if isinstance(h, int):
                bloque = tipo_bloque_especial(nivel, d, h)
                if bloque:
                    tabla.at[h, d] = bloque
            else:
                tabla.at[h, d] = "TUTORIA"

    # Insertar materias reales
    datos = df[(df["Nivel"] == nivel) & (df["Aula"] == aula)]
    datos_unicos = datos.groupby(["Hora", "Día"], as_index=False).first()
    for _, fila in datos_unicos.iterrows():
        hora = int(fila["Hora"])
        dia = fila["Día"]
        materia = latex_escape(fila["Materia"])
        docente = latex_escape(fila["Docente"])
        tabla.at[hora, dia] = f"\\makecell{{{materia}\\\\({docente})}}"

    # Construcción del bloque LaTeX
    latex = []
    latex.append("\\begin{landscape}")
    latex.append("\\begin{table}[H]")
    latex.append("\\centering")
    latex.append("\\scriptsize")
    latex.append(f"\\caption{{Horario para {latex_escape(nivel)} {latex_escape(aula)}}}")
    latex.append("\\setlength{\\arrayrulewidth}{1pt}")
    latex.append("\\arrayrulecolor{black}")
    latex.append("\\begin{tabular}{|c|c|" + "c|" * len(dias) + "}")
    latex.append("\\specialrule{1pt}{0pt}{0pt}")
    latex.append("\\rowcolor{white}")
    latex.append("\\textbf{Hora} & \\textbf{Horario} & " + " & ".join([f"\\textbf{{{d}}}" for d in dias]) + " \\\\ \\specialrule{1pt}{0pt}{0pt}")

    for h in horas:
        fila = [str(h), franjas[h]]
        for d in dias:
            celda = tabla.at[h, d]
            if isinstance(celda, str) and celda in colores:
                fila.append(f"\\cellcolor{{{colores[celda]}}}\\textbf{{{celda}}}")
            else:
                fila.append(celda if pd.notna(celda) else "")
        latex.append(" & ".join(fila) + " \\\\ \\hline")

    latex.append("\\end{tabular}")
    latex.append(f"\\label{{tab:{nivel}_{aula}}}")
    latex.append("\\end{table}")
    latex.append("\\end{landscape}")
    return "\n".join(latex)


### ▶️ 5. Ejemplo de ejecución para "1EGB - Exitosos"

Se genera e imprime la tabla LaTeX del horario correspondiente al aula "Exitosos" del nivel "1EGB".


In [ ]:
# --- 5. Ejecutar para ejemplo: 1EGB Ejemplares ---
latex_code = generar_tabla_latex("1EGB", "Exitosos", df)
print(latex_code)

\begin{landscape}
\begin{table}[H]
\centering
\scriptsize
\caption{Horario para 1EGB Exitosos}
\setlength{\arrayrulewidth}{1pt}
\arrayrulecolor{black}
\begin{tabular}{|c|c|c|c|c|c|c|}
\specialrule{1pt}{0pt}{0pt}
\rowcolor{white}
\textbf{Hora} & \textbf{Horario} & \textbf{Lunes} & \textbf{Martes} & \textbf{Miércoles} & \textbf{Jueves} & \textbf{Viernes} \\ \specialrule{1pt}{0pt}{0pt}
Act. Inicial & 7h30 a 7h45 & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} \\ \hline
1 & 7h45 a 8h15 & \makecell{LEN\_1EGB\\(MALENA OÑA)} & \makecell{LEC\_1EGB\\(ALISON PLAZA)} & \cellcolor{yellow!30}\textbf{EVALEV} & \makecell{MAT\_1EGB\\(PAULINA ÑACATO)} & \makecell{EST\_1EGB\\(SANTIAGO CALLE)} \\ \hline
2 & 8h15 a 8h45 & \makecell{LEN\_1EGB\\(MALENA OÑA)} & \makecell{ING\_1EGB\\(MARCIA HERRERA VÁSQUEZ)} & \cellcolor{yellow!30}\textbf{EVALEV} & \makecell{M

### 📑 6. Impresión de todos los horarios ordenados

La función `imprimir_todos_los_horarios_ordenados(df)` genera las tablas LaTeX para todos los niveles y aulas presentes en el DataFrame `df`.

- Se define un orden lógico para los niveles educativos (de 1EGB a 3BGU).
- Se eliminan duplicados y se ordenan los pares (Nivel, Aula).
- Se recorre cada combinación ordenada y se imprime la tabla LaTeX generada con la función `generar_tabla_latex`.

Este bloque permite generar automáticamente todos los horarios del sistema en un solo paso, siguiendo una estructura ordenada y lista para ser copiada o insertada en un documento de LaTeX.


In [ ]:
def imprimir_todos_los_horarios_ordenados(df):
    orden_niveles = ['1EGB', '2EGB', '3EGB', '4EGB', '5EGB', '6EGB',
                     '7EGB', '8EGB', '9EGB', '10EGB', '1BGU', '2BGU', '3BGU']

    niveles_aulas = df[["Nivel", "Aula"]].drop_duplicates()

    # Asignar un valor de orden a cada nivel
    niveles_aulas["Orden"] = niveles_aulas["Nivel"].apply(lambda x: orden_niveles.index(x) if x in orden_niveles else -1)

    # Ordenar por el índice definido
    niveles_aulas = niveles_aulas.sort_values(by=["Orden", "Aula"])

    for _, fila in niveles_aulas.iterrows():
        nivel = fila["Nivel"]
        aula = fila["Aula"]
        print(f"% === HORARIO: {nivel} {aula} ===")
        print(generar_tabla_latex(nivel, aula, df))
        print("\n\n")

imprimir_todos_los_horarios_ordenados(df)


% === HORARIO: 1EGB Ejemplares ===
\begin{landscape}
\begin{table}[H]
\centering
\scriptsize
\caption{Horario para 1EGB Ejemplares}
\setlength{\arrayrulewidth}{1pt}
\arrayrulecolor{black}
\begin{tabular}{|c|c|c|c|c|c|c|}
\specialrule{1pt}{0pt}{0pt}
\rowcolor{white}
\textbf{Hora} & \textbf{Horario} & \textbf{Lunes} & \textbf{Martes} & \textbf{Miércoles} & \textbf{Jueves} & \textbf{Viernes} \\ \specialrule{1pt}{0pt}{0pt}
Act. Inicial & 7h30 a 7h45 & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} & \cellcolor{purple!20}\textbf{TUTORIA} \\ \hline
1 & 7h45 a 8h15 & \makecell{MAT\_1EGB\\(PAULINA ÑACATO)} & \makecell{LEN\_1EGB\\(MALENA OÑA)} & \cellcolor{yellow!30}\textbf{EVALEV} & \makecell{ING\_1EGB\\(MARCIA HERRERA VÁSQUEZ)} & \makecell{LEN\_1EGB\\(MALENA OÑA)} \\ \hline
2 & 8h15 a 8h45 & \makecell{MAT\_1EGB\\(PAULINA ÑACATO)} & \makecell{LEN\_1EGB\\(MALENA OÑA)} & \cellcolor{yell